In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ Enabled memory growth for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(e)


2026-02-18 11:45:56.759749: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-18 11:45:56.787051: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-18 11:45:56.787072: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-18 11:45:56.787096: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-18 11:45:56.792964: I tensorflow/core/platform/cpu_feature_g

✅ Enabled memory growth for 1 GPU(s)


2026-02-18 11:45:58.502895: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-18 11:45:58.519869: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-18 11:45:58.521810: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [3]:
from utils.data import load_data

dataset = load_data()

In [4]:
dataset.keys()

X_train = dataset["X_train"]
y_train = dataset["y_train"]

X_test= dataset["X_test"]
y_test = dataset["y_test"]

labels = dataset["labels"]

n_labels = len(labels)


In [5]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
X_train.shape

(50000, 32, 32, 3)

In [20]:
tf.keras.backend.clear_session()

In [11]:
# One-hot encode labels
y_train_cat = to_categorical(y_train, n_labels)
y_test_cat = to_categorical(y_test, n_labels)

In [21]:
DATASET_SIZE = len(X_train)  # Total number of examples
train_size = int(0.8 * DATASET_SIZE)
val_size = int(0.2 * DATASET_SIZE)

# 2. Shuffle and split the base dataset (BEFORE batching)
# Important: Shuffle first so you don't get all one class in validation
full_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_cat))
full_dataset = full_dataset.shuffle(buffer_size=1000, seed=42)

train_dataset = full_dataset.take(train_size)
val_dataset = full_dataset.skip(train_size)


def process_batch(image, label, new_size = (96, 96)):
    image = tf.image.resize(image, new_size)
    image = preprocess_input(image) 
    return image, label

BATCH_SIZE = 32

# 3. Apply your batching/preprocessing pipeline to BOTH
def process_pipeline(ds):
    return (
        ds
        .batch(BATCH_SIZE)
        .map(process_batch, num_parallel_calls=tf.data.AUTOTUNE)
        .prefetch(tf.data.AUTOTUNE)
    )

train_ds = process_pipeline(train_dataset)
val_ds = process_pipeline(val_dataset)


2026-02-18 12:05:16.529805: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 12.0
2026-02-18 12:05:16.529823: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2026-02-18 12:05:16.529855: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-18 12:05:16.529861: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-18 12:05:16.529883: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2026-02-18 12:05:16.529899: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:19

In [15]:
# # tf.expand_dims(X_train, -1)
# #Adds an extra dimension at the end to convert the data shape from (batch_size, 28, 28)
# #to (batch_size, 28, 28, 1), which is needed because TensorFlow image functions expect a channel dimension. Here, '1' represents the single grayscale channel.
# X_train_resized = tf.image.resize(X_train, (96, 96))
# # X_test_resized = tf.image.resize(X_test, (96, 96))

# #Converts the grayscale image with shape (96, 96, 1) into a 3-channel RGB image (96, 96, 3)
# #by duplicating the single grayscale channel three times. MobileNetV2 expects 3 channels (RGB),
# #so this step adapts the Fashion MNIST grayscale images to that format.

# # X_train_rgb = tf.image.grayscale_to_rgb(X_train_resized)
# # X_test_rgb = tf.image.grayscale_to_rgb(X_test_resized)

# # Convert to numpy and preprocess for MobileNetV2
# #converts these tensors to NumPy arrays, which some preprocessing functions require.
# X_train_rgb = preprocess_input(X_train_resized)
# # X_test_rgb = preprocess_input(X_test_resized)


In [16]:
# Build transfer learning model
base_model = MobileNetV2(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base

9406464/9406464 [==============================] - 0s 0us/step


In [17]:

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), # Pool spatial features into a vector
    layers.Dense(128, activation='relu'), # Fully connected layer to learn new patterns
    layers.Dense(n_labels, activation='softmax') # Output layer with 10 classes (Fashion MNIST)
])

In [18]:
# Compile model
# Compile the model with Adam optimizer and categorical crossentropy loss

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
# Train
# history = model.fit(X_train_rgb, y_train_cat, validation_split=0.2, epochs=5, batch_size=64)

history = model.fit(train_ds, validation_data=val_ds, epochs=5, batch_size=64)

Epoch 1/5


2026-02-18 12:05:38.559859: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2026-02-18 12:06:18.405756: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


   1/1250 [..............................] - ETA: 20:04:57 - loss: 3.1981 - accuracy: 0.1875

2026-02-18 12:06:29.850167: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1b0f179b70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-02-18 12:06:29.850188: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 5070 Ti, Compute Capability 12.0
2026-02-18 12:06:29.852373: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-02-18 12:06:29.869561: W tensorflow/compiler/xla/service/gpu/nvptx_compiler.cc:445] Couldn't read CUDA driver version.
2026-02-18 12:06:29.877649: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1250/1250 [==============================] - 66s 6ms/step - loss: 0.5173 - accuracy: 0.8239 - val_loss: 0.4077 - val_accuracy: 0.8563
Epoch 2/5
1250/1250 [==============================] - 6s 5ms/step - loss: 0.3656 - accuracy: 0.8731 - val_loss: 0.4072 - val_accuracy: 0.8577
Epoch 3/5
1250/1250 [==============================] - 6s 5ms/step - loss: 0.3036 - accuracy: 0.8952 - val_loss: 0.4092 - val_accuracy: 0.8595
Epoch 4/5
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2556 - accuracy: 0.9106 - val_loss: 0.4115 - val_accuracy: 0.8589
Epoch 5/5
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2112 - accuracy: 0.9270 - val_loss: 0.4484 - val_accuracy: 0.8554


In [ ]:
from tensorflow import keras
import tensorflow as tf
import gc

# After training/inference
keras.backend.clear_session()
gc.collect()
